In [1]:
## script to generate Scispot registrations for ChESS 96-well plate
## lkp 2022/11/15

import pandas as pd
import requests
import json
import sys

sys.stdout.write("Imported required packages")

Imported required packages

In [9]:
api_key_file = 'G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/scispot_api_key.txt'
API_KEY = open(api_key_file, 'r').readlines()[0]
print(API_KEY)

3F4E0776-EFEF-4A5B-8479-01B7C2367090


# Jupyter notebook in Scispot

- Scispot --> LabSheets --> Jupyter --> +New --> write script/notebook
- - need jupyterhub server credentials?

- ? can I look up a UUID by sample name?
- - update-by-id to fetch UUID from Sample ID
- ? can I look up UUIDs by manifest name?
- - not yet! 
- ? can I specify other "variable" inputs/params? or can scientist edit the ipynb with those values?
- - A: yes, can code input parameters 
- - ? can I read in other sheets into pandas?
- - - nm don't worry about this for now...


### tips
- update-by-id to fetch UUID from Sample ID 
- https://www.loom.com/share/32c01c2cb1294720b15596b0a43af5e9

In [10]:
# lookup an entry by UUID
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

test_uuid = "0344a99d-888a-443e-9a23-13eceb279ae1"
find_entry_from_uuid('Sample Manager', test_uuid)

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'row': ['0344a99d-888a-443e-9a23-13eceb279ae1',
  'THP1_dbgen1_VTP50469_3',
  'VTP50469_1',
  'THP1_dbgen1',
  '',
  '',
  '',
  '',
  '',
  'Frozen Nuclei',
  '90b19820-726d-4086-b74a-199d37899ea5',
  '',
  'ChESS_dbgen1_202212_plate1rep3',
  '',
  '',
  '',
  '',
  ''],
 'success': 'true'}

In [4]:
# turn json into pandas df
def json_to_pd_df(json_in):
    rows = json_in["row"]
    column_labels = json_in["headers"]
    df_out = pd.DataFrame(data=rows, columns=column_labels)
    return df_out

temp = find_entry_from_uuid('Sample Manager', test_uuid)
temp_df = pd.DataFrame(data=find_entry_from_uuid('Sample Manager', test_uuid)['row']).T
temp_df.columns = list(temp['headers'])
temp_df

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,0344a99d-888a-443e-9a23-13eceb279ae1,THP1_dbgen1_VTP50469_3,VTP50469_1,THP1_dbgen1,,,,,,Frozen Nuclei,90b19820-726d-4086-b74a-199d37899ea5,,ChESS_dbgen1_202212_plate1rep3,,,,0.000,


In [11]:
# fetch any row from the Protein Manager to get the columns
def fetch_random_entry():
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Protein Manager",
        "pageSize": "1",
        "page": "1"
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# initialize an empty Protein Manager
prot_df = pd.DataFrame(data=fetch_random_entry()['rows'], columns=fetch_random_entry()['headers'])
prot_df= prot_df.drop(prot_df.index[0])
prot_df

,UUID,Protein ID,Sample ID,Protocol ID,Extraction Date,Extraction Scientist,Extraction Fraction,Extraction Notes,Protein Concentration (ug/mL),BCA Results,Location,Storage Form,Storage Amount (uL),Pass,Parent,Children,Created By,Creation Date


In [12]:
def fetch_entry_from_id(manager, sample_id):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row-by-id"
    payload = {
        "apiKey": API_KEY,
        "labsheet": manager,
        "id": sample_id
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

test_sampleid = "THP1_dbgen1_VTP50469_3"
fetch_entry_from_id("Sample Manager",test_sampleid)

{'rows': [['0344a99d-888a-443e-9a23-13eceb279ae1',
   'THP1_dbgen1_VTP50469_3',
   'VTP50469_1',
   'THP1_dbgen1',
   '',
   '',
   '',
   '',
   '',
   'Frozen Nuclei',
   '90b19820-726d-4086-b74a-199d37899ea5',
   '',
   'ChESS_dbgen1_202212_plate1rep3',
   '',
   '',
   '',
   '',
   ''],
  ['83d5ede7-4fb0-4659-8b74-a0ed01e63175',
   'THP1_dbgen1_VTP50469_3',
   'VTP50469_1',
   'THP1_dbgen1',
   '',
   '',
   '',
   '',
   '',
   '',
   '90b19820-726d-4086-b74a-199d37899ea5',
   '',
   'ChESS_dbgen1_202212_plate1rep3',
   '',
   '',
   '',
   '0.000',
   '']],
 'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'success': True}

In [38]:
# add a new row to the Protein Manager
def add_entry1(new_row):
    print(new_row)
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    ret = session.post(url, json=payload)
    print(response.text)

def add_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    print(new_row)
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
# to add rows to Sample Manager, some columns required:
test_new_row_this_works = [
      "proteinid_here",
      "0344a99d-888a-443e-9a23-13eceb279ae1", # Sample ID required, must be UUID
      "e45d66e9-abed-4d24-9dd8-8f074233c2d8", # Protocol ID required, must be UUID
      "01/01/2023", # "Extraction Date" column required, must be MM/DD/YYYY
      "Lindsay", # "Extraction Scientist" column required, must be name
      "extractionfrx_here",
      "extractionnote_here",
      "proteinconc_here",
      "bca_here",
      "location_here",
      "storageform_here",
      "storageamt_here",
      "False", # "Pass" column required
      "parent_here",
      "children_here",
      "createdby_here",
     "creationdate_here"
]
    
#add_entry(test_new_row_this_works)

In [25]:
import datetime

## sample specific variables
# name the Protein ID after the Sample ID
prot_df['Sample ID'] = temp_df['Sample ID']
prot_df['Protein ID'] = temp_df['Sample ID'] + "_chromatin"

# link the Parent Sample ID
prot_df['Parent'] = temp_df['UUID']

## batch specific variables
extraction_date = "12/01/2022"
extraction_scientist = "Lindsay"
extraction_fraction = "chromatin"
created_by = "Lindsay"
creation_date = datetime.datetime.now().strftime("%x")
prot_df['Extraction Date'] = extraction_date
prot_df['Extraction Scientist'] = extraction_scientist
prot_df['Extraction Fraction'] = extraction_fraction
prot_df['Created By'] = created_by
prot_df['Creation Date'] = creation_date


# extraction scientist

list(prot_df.iloc[0][1:])

['THP1_dbgen1_VTP50469_3_chromatin',
 'THP1_dbgen1_VTP50469_3',
 nan,
 '12/01/2022',
 'Lindsay',
 'chromatin',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '0344a99d-888a-443e-9a23-13eceb279ae1',
 nan,
 'Lindsay',
 '03/04/23']

In [73]:
## TODO check if row added? THP1_dbgen1_VTP50469_3
# uuids that geenrated from add_entry to protein manager:
# 0344a99d-888a-443e-9a23-13eceb279ae1 - Sample Manager and Protein Manager and Peptide Manager and ...??
# 90b19820-726d-4086-b74a-199d37899ea5 - Sample Manager and Protein Manager and Peptide Manager and ...??
# 98f8e123-84ab-4171-b26e-6c3c0ad6ecf3 - Sample Manager and Protein Manager and Peptide Manager and ...??
# f83d6043-0901-4362-bef9-324987e06451 - doesn't exist

find_entry_from_uuid("Runs Manager", "0344a99d-888a-443e-9a23-13eceb279ae1")

{'headers': ['UUID',
  'Run ID',
  'Digestion ID',
  'Acquisition ID',
  'Pass',
  'Orig Location',
  'S3 Location',
  'Method',
  'Injection (uL)',
  'Injection (ug)',
  'Exp Condition',
  'Acquisition Type',
  'Acquisition Position',
  'Created By'],
 'row': ['0344a99d-888a-443e-9a23-13eceb279ae1',
  '',
  '',
  '',
  '',
  'ChESS_dbgen1_202212_plate1rep3',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 'success': 'true'}

In [ ]:
# someday I should actually make objects for scispot stuff...
# TODO: read in headers from API? so that if they change, this isn't all hardcoded...
class scispot_managers:
    def __init__( self, data, ):
        self.sample = pd.DataFrame()
        self.protein = pd.DataFrame()
        self.peptide = pd.DataFrame()
        
    def sample_to_protein( self, input ):
        self.peptide[ 'new_column' ] = input

def initiate_sample_df():
    df = pd.DataFrame(columns=['Sample ID',
                                 'Working compound',
                                 'Working cell line',
                                 'Pert Dose',
                                 'Pert Time',
                                 'Pert Protocol',
                                 'Tissue ID',
                                 'Tissue Passage',
                                 'Type',
                                 'Storage Form',
                                 'Storage Amount',
                                 'Pass',
                                 'Quantity Number of Cells (x1000)',
                                 'Created By'])
    return(df)

def initiate_protein_df(sample_df, frx):
    df = pd.DataFrame(columns=['Protein ID',
                                 'Sample ID',
                                 'Protocol ID',
                                 'Extraction Date',
                                 'Extraction Scientist',
                                 'Extraction Fraction',
                                 'Extraction Notes',
                                 'Protein Concentration (ug/mL)',
                                 'BCA Results',
                                 'Location',
                                 'Storage Form',
                                 'Storage Amount (uL)',
                                 'Pass',
                                 'Created By',
                                 'Creation Date'])
    
    df['Sample ID'] = sample_df['Sample ID']
    df['Protein ID'] = sample_df['Sample ID'].astype(str) + '_' + frx
    df['Extraction Fraction'] = frx
    # TODO add datetime ['Creation Date']
    # TODO add log-in ['Created By']
    # TODO inherit Sample ID UUID from Sample ID 
    # TODO add 
    
    return(df)

def initiate_peptide_df(input_df):
    df = pd.DataFrame(columns=['Digestion ID',
                                 'Protein ID',
                                 'Digestion Date',
                                 'Protocol ID',
                                 'Digestion Scientist',
                                 'Protein Input',
                                 'Reduction Chemical',
                                 'Reduction Dose',
                                 'Alkylation Chemical',
                                 'Alkylation Dose',
                                 'Protease Chemical',
                                 'Protease Dose',
                                 'Quench Chemical',
                                 'Quench Dose',
                                 'Storage Location',
                                 'Storage Form',
                                 'Storage Concentration',
                                 'Pass',
                                 'Location',
                                 'Created By'])
    
    df['Protein ID'] = input_df['Protein ID']
    df['Digestion ID'] = input_df['Protein ID'].astype(str) + '_peptide'
    # add datetime ['Creation Date']
    # add log-in ['Created By']
    
    # populate some default values
    df['Reduction Chemical'] = 'DTT'
    df['Alkylation Chemical'] = 'IAA'
    df['Protease Chemical'] = 'Trypsin'
    
    return(df)

## TODO: what columns to inherit from Sample? Plate position? populate default values?

In [ ]:
# read in a Scispot plate TSV export of Sample Manager
# note: export doesn't include UUID
## TODO possible to pull a plate of samples with some manifest name/location from API?
sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-03-02T23_27_01.tsv"
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.head()


In [ ]:
## GENERATE PROTEIN SAMPLES
## TODO: this is pretty inelegant, could perhaps reduce this to a list of fractions to generate?

# generate protein samples: cytosol
protein_df_cyto = initiate_protein_df(input_df, 'Cytosolic')

# generate protein samples: nucleoplasm
protein_df_nucleo = initiate_protein_df(input_df, 'Nucleoplasm')

# generate protein samples: euchromatin
protein_df_euch = initiate_protein_df(input_df, 'Euchromatin')

# generate protein samples: heterochromatin
protein_df_het = initiate_protein_df(input_df, 'Heterochromatin')

# generate protein samples: insoluble
protein_df_ins = initiate_protein_df(input_df, 'Insoluble')

# append all protein to one protein dataframe
protein_df = protein_df_cyto.append(protein_df_nucleo.append(protein_df_euch.append(protein_df_het.append(protein_df_ins))))

# 


In [ ]:

print(protein_df.head())
print(peptide_df.head())